# Post-Training Quantization in PyTorch using the Model Compression Toolkit (MCT)
[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/mct_features_notebooks/pytorch/example_pytorch_post_training_quantization.ipynb)

## Overview
This quick-start guide explains how to use the **Model Compression Toolkit (MCT)** to quantize a PyTorch model. We will load a pre-trained model and  quantize it using the MCT with **Post-Training Quatntization (PTQ)**. Finally, we will evaluate the quantized model and export it to an ONNX file.

## Summary
In this tutorial, we will cover:

1. Loading and preprocessing ImageNet’s validation dataset.
2. Constructing an unlabeled representative dataset.
3. Post-Training Quantization using MCT.
4. Accuracy evaluation of the floating-point and the quantized models.

## Setup
Install the relevant packages:

In [1]:
!pip install -q torch torchvision


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import importlib
if not importlib.util.find_spec('model_compression_toolkit'):
    !pip install model_compression_toolkit

In [5]:
import torch
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from torchvision.datasets import ImageNet
import numpy as np
import random
from ultralytics  import YOLO

Load a pre-trained MobileNetV2 model from torchvision, in 32-bits floating-point precision format.

In [9]:
float_model = YOLO('plume.pt')

## Dataset preparation
### Download ImageNet validation set
Download ImageNet dataset with only the validation split.

**Note** that for demonstration purposes we use the validation set for the model quantization routines. Usually, a subset of the training dataset is used, but loading it is a heavy procedure that is unnecessary for the sake of this demonstration.

This step may take several minutes...

In [55]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader

from PIL import Image
from io import BytesIO
import io


Extract ImageNet validation dataset using torchvision "datasets" module.

In [111]:
splits = {
    'train': 'hf://datasets/edouard-rolland/volcanic-plumes/data/train-00000-of-00001.parquet',
    'validation': 'hf://datasets/edouard-rolland/volcanic-plumes/data/validation-00000-of-00001.parquet',
    'test': 'hf://datasets/edouard-rolland/volcanic-plumes/data/test-00000-of-00001.parquet'
}

# Load each split into a DataFrame
train_df = pd.read_parquet(splits['train'])
validation_df = pd.read_parquet(splits['validation'])
test_df = pd.read_parquet(splits['test'])



from torchvision import transforms
from PIL import Image
import io

# Define the transformation to convert PIL images to tensors
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size (optional)
    transforms.ToTensor()  # Convert the PIL Image to a tensor
])

import pandas as pd
from PIL import Image
import io
from torchvision import transforms

class VolcanicPlumeDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, transform=None):
        # Ensure that the dataframe is a pandas DataFrame
        if not isinstance(dataframe, pd.DataFrame):
            raise TypeError(f"Expected dataframe to be a pandas DataFrame, got {type(dataframe)}")
        
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        print(type(self.dataframe))  # Check if it's a pandas DataFrame
        row = self.dataframe.iloc[idx]
        image_bytes = row['image']['bytes']
        image = Image.open(io.BytesIO(image_bytes))  # Convert bytes to PIL Image

        # Apply transformation
        if self.transform:
            image = self.transform(image)

        label = row['objects']
        return image, label

import pandas as pd
from torch.utils.data import DataLoader

# Load your dataset as a pandas DataFrame (this could be from a CSV, Parquet file, etc.)
dataframe = pd.read_parquet('/path/to/your/dataset.parquet')  # Example: replace with your actual file path

# Now create the VolcanicPlumeDataset using the dataframe
train_dataset = VolcanicPlumeDataset(dataframe, transform=transform)

# Create a DataLoader to load the data in batches
batch_size = 16
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

train_dataset = VolcanicPlumeDataset(train_df)

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/your/dataset.parquet'

## Representative Dataset
For quantization with MCT, we need to define a representative dataset required by the PTQ algorithm. This dataset is a generator that returns a list of images:

In [110]:
train_dataset = VolcanicPlumeDataset(train_dataset, transform=transform)


# Now, you can create the DataLoader
from torch.utils.data import DataLoader
batch_size = 16
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

import torch
from torchvision import transforms

# Define a transformation to convert the image to a tensor
transform = transforms.ToTensor()

def representative_dataset_gen():
    dataloader_iter = iter(dataloader)
    for _ in range(n_iter):
        image, label = next(dataloader_iter)  # The image is already a tensor
        yield [image]




TypeError: Expected dataframe to be a pandas DataFrame, got <class '__main__.VolcanicPlumeDataset'>

## Target Platform Capabilities (TPC)
In addition, MCT optimizes the model for dedicated hardware platforms. This is done using TPC (for more details, please visit our [documentation](https://sony.github.io/model_optimization/docs/api/api_docs/modules/target_platform.html)). Here, we use the default Pytorch TPC:

In [73]:
import model_compression_toolkit as mct

# Get a TargetPlatformCapabilities object that models the hardware platform for the quantized model inference. Here, for example, we use the default platform that is attached to a Pytorch layers representation.
target_platform_cap = mct.get_target_platform_capabilities('pytorch', 'default')

## Post-Training Quantization using MCT
Now for the exciting part! Let’s run PTQ on the model. 

In [74]:
quantized_model, quantization_info = mct.ptq.pytorch_post_training_quantization(
        in_module=float_model,
        representative_data_gen=representative_dataset_gen,
        target_platform_capabilities=target_platform_cap
)

AttributeError: 'VolcanicPlumeDataset' object has no attribute 'iloc'

Our model is now quantized. MCT has created a simulated quantized model within the original PyTorch framework by inserting [quantization representation modules](https://github.com/sony/mct_quantizers). These modules, such as `PytorchQuantizationWrapper` and `PytorchActivationQuantizationHolder`, wrap PyTorch layers to simulate the quantization of weights and activations, respectively. While the size of the saved model remains unchanged, all the quantization parameters are stored within these modules and are ready for deployment on the target hardware. In this example, we used the default MCT settings, which compressed the model from 32 bits to 8 bits, resulting in a compression ratio of 4x. Let's print the quantized model and examine the quantization modules:

## Model Evaluation
In order to evaluate our models, we first need to load the validation dataset. 

In [ ]:
val_dataloader = DataLoader(dataset, batch_size=50, shuffle=False, num_workers=16, pin_memory=True)

Now, we will create a function for evaluating a model.

In [ ]:
from tqdm import tqdm


def evaluate(model, testloader):
    """
    Evaluate a model using a test loader.
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # correct += (predicted == labels).sum().item()
    val_acc = (100 * correct / total)
    print('Accuracy: %.2f%%' % val_acc)
    return val_acc

Let's start with the floating-point model evaluation.

In [ ]:
evaluate(float_model, val_dataloader)

Finally, let's evaluate the quantized model:

In [ ]:
evaluate(quantized_model, val_dataloader)

You can see that we got a very small degradation with a compression rate of x4 !
Now, we can export the quantized model to ONNX:

In [ ]:
mct.exporter.pytorch_export_model(quantized_model, save_model_path='qmodel.onnx', repr_dataset=representative_dataset_gen)

## Conclusion

In this tutorial, we demonstrated how to quantize a classification model for MNIST in a hardware-friendly manner using MCT. We observed that a 4x compression ratio was achieved with minimal performance degradation.

The key advantage of hardware-friendly quantization is that the model can run more efficiently in terms of runtime, power consumption, and memory usage on designated hardware.

MCT can deliver competitive results across a wide range of tasks and network architectures. For more details, [check out the paper:](https://arxiv.org/abs/2109.09113).

## Copyrights

Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
